## AnkiPy

Python library for creating and managing Anki decks using an excel file as database of cards

IMPORTANTE IMÁGENES: Si tu usuario de Windows se llama "Juan" y tu perfil de Anki se llama "Estudios", la ruta para copiar las imágenes sería:

C:\Usuarios\Juan\AppData\Roaming\Anki2\Estudios\collection.media


Author: Mario Mañana     
Version log:    
03/10/2024 Minor updates for increasing the functionality.   
31/08/2024 Include a setting option for Electrical Machines and Drives.    
28/08/2024 Fully functional code for importing both Circuit Theory and UC24 databases.    


packages

In [78]:
import pandas as pd
import os
import re

### Flags

In [79]:
# flags
# use .- Set target: 1.-  Proyecto de innovacion docente Teoría de Circuitos; 2.- UC24; 3.- Proyecto de innovación docente Máquinas y Accionamientos Eléctricos
use = 3 # subjetc G875

# asignatura. ie: set 'uc24' for UC 24 or Gxxx for the subject (IMPORTANT: link to the right excel file)  
asignatura = 'G875' #'G861' # 'G875' # 'G990' #G589-G620'  # 'G822'


baggregated = True

boverwrite = False 

if use == 1: # innovación docente. Circuit Theory
    path = 'E:\\mario\\trabajos2\\innovación_docente_2024\\ankipy\\'
    excel_file = 'uc_tc.xlsx'
    csv_out = 'uc_tc.csv' # default name
    sheet_names = ['intro', 'ca', 'resolucion', 'trifasica', 'transitorio', 'cuadripolos', 'bobinas', 'filtros', 'rns']
    #sheet_names = ['intro', 'ca']
elif use == 2: # uc24
    path = 'E:\\mario\\UC24\\documentacion\\'
    excel_file = 'uc24_anki.xlsx'
    csv_out = 'uc24.csv'
    sheet_names = ['eadmi', 'presupuesto', 'infraestructuras', 'LOSU', 'academico', 'CifrasUC']        # Name of the sheets to read
    #sheet_names = ['eadmi']        # Name of the sheets to read
elif use == 3: # innovación docente. Electrical machines and drives
    path = 'E:\\mario\\trabajos2\\innovación_docente_2024\\ankipy\\'
    excel_file = 'uc_mae.xlsx'
    csv_out = 'uc_mae.csv' # default name
    sheet_names = ['intro', 'mecanica', 'convertidores', 'ca', 'cc', 'reluctancia']
else: # unknown value
    print('Set target: error. Unknown value...')


# columns
columns = ['slide', 'pregunta', 'imagen_frontal', 'respuesta', 'imagen_respuesta', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9', 'r10', 'tags','deck',asignatura] # List of columns to read

subdesk_uc24 = {
    "1": "eAdministracion",
    "2": "Presupuesto",
    "3": "Infraestructuras",
    "4": "LOSU",
    "5": "Academico",
    "6": "CifrasUC"
}

subdesk_mae = {
    "1": "Introduction",
    "2": "Mechanical requirements",
    "3": "Power converters",
    "4": "ac drives",
    "5": "dc drives",
    "6": "Switched-reluctance drives"
}


csv_aggregated = asignatura + '.csv'

full_excel_path = path + excel_file
print("******************************************")
print(" Input File")
print( full_excel_path)
print("  ")


if baggregated == True:
    full_csv_output = path + csv_aggregated
else:
    full_csv_output = path + sheet + '_00.csv'
print('Output file: ' + full_csv_output)

******************************************
 Input File
E:\mario\trabajos2\innovación_docente_2024\ankipy\uc_mae.xlsx
  
Output file: E:\mario\trabajos2\innovación_docente_2024\ankipy\G875.csv


In [80]:
# File header
# More information: https://docs.ankiweb.net/importing/text-files.html 
contenido = """#deck column:17
#separator:Semicolon
#notetype:Basic_Image_several_answers
#columns:slide;pregunta;imagen_frontal;respuesta;imagen_respuesta;r1;r2;r3;r4;r5;r6;r7;r8;r9;r10;tags;deck
#tags column:16
#html:true
"""

# Escribir (sobrescribir si ya existe) el archivo
with open( full_csv_output, "w") as archivo:
    archivo.write(contenido)

# Iterate over the sheet_names array
firsti = True
df_summary = pd.DataFrame(columns=['Subdesk', 'NCards'])

for index, sheet in enumerate( sheet_names):
    #print("-------------------")
    #print("Sheet: " + sheet)
    
    
    df = pd.read_excel( full_excel_path, sheet_name=sheet, usecols=columns, header=0, dtype=str)
    df.columns = df.columns.astype( str)
    
    # remove rows which <asignatura> cell is NaN
    df2 = df[ df[asignatura].notna()]
    
    # set <slide> and <deck> cells   
    for index, row in df2.iterrows():
        df2.at[ index, 'slide'] = sheet + '.' + str(row['slide'])
        if use == 1: # Teoría de Circuitos
            df2.at[ index, 'deck'] = asignatura + '::' + 'Tema ' + str(row[asignatura])  
        elif use == 2: # UC24
            df2.at[ index, 'deck'] = asignatura + '::' + subdesk_uc24[ row[asignatura]]
            #subdesk = subdesk_uc24     
        elif use == 3: # Máquinas y Accionamientos eléctricos
            if asignatura == 'G875':
                #df2.at[ index, 'deck'] = asignatura + "::" + subdesk_mae[ row[asignatura]]   
                df2.at[ index, 'deck'] = asignatura + "::" + str( row[asignatura])   
                #subdesk = subdesk_mae
            else:
                df2.at[ index, 'deck'] = asignatura + '::' + '' + str(row[asignatura])  
        else: #unknown value
            print('Set target: Unknown value...')    
    
           
    #nueva_fila = {'Subdesk': [subdesk[ row[asignatura]]], 'NCards': [len(df2)]}
    #df_nuevo = pd.DataFrame( nueva_fila)
    #df_summary = pd.concat([df_summary, df_nuevo], ignore_index=True)
    
        #df_nuevo = pd.DataFrame({
        #    'Subdesk': subdesk_uc24[ row[asignatura]], 
        #    'NCards': len(df2)})
        #df_summary = pd.concat([df_summary, df_nuevo], ignore_index=True)       
    # remove <asignatura> column once the dataframe has been filtered   
    dfs = df2.drop( columns=[asignatura])
    dfs.to_csv( full_csv_output, sep=';', index=False, na_rep='  ', mode='a', header=False)
       
    # aggregate all sheets in a single dataframe
    if firsti == True: 
        dfall = dfs
        firsti = False
    else:
        dfall = pd.concat([dfall, dfs], ignore_index=True)
    
    #print(dfs)
    
   
#dfall es el DataFrame original y 'deck' es el nombre de la columna que contiene los valores str
conteo_valores = dfall['deck'].value_counts().reset_index()
# Renombrar las columnas para que sean descriptivas
conteo_valores.columns = ['Subdesk', 'NCards']
# Imprimir el número de tarjetas por mazo
#print(conteo_valores)

    
# Determinar el ancho de las columnas basándote en los valores más largos
ancho_subdesk = max(len(subdesk) for subdesk in conteo_valores['Subdesk']) + 10  # Ancho de la columna 'Subdesk'
ancho_cards = max(len(str(card)) for card in conteo_valores['NCards']) + 5  # Ancho de la columna 'Cards'

print(f"{'Subdesk':<{ancho_subdesk}}{'Cards':<{ancho_cards}}")
print('-' * (ancho_subdesk + ancho_cards + 3))

for subdesk, cards in zip( conteo_valores['Subdesk'], conteo_valores['NCards']):
    print(f"{subdesk:<{ancho_subdesk}}{cards:<{ancho_cards}}")

Subdesk                                Cards  
-------------------------------------------------
G875::Power Converters                 14     
G875::Stepper Motor                    4      
G875::Introduction                     1      
G875::Mechanical requirements          1      


In [81]:

# Definir una función para extraer nombres de archivos después de 'img src="'
def extract_filenames(text):
    # Expresión regular para encontrar nombres de archivos entre comillas después de 'img src="'
    #return re.findall(r'img src="([^"]+)"', text)
    #return re.findall(r"img src='([^']+)'", text)
    #return re.findall(r'img src="([^"]+)"', text)

    # Use a regular expression to find the text between the quotes after 'img src='
    match = re.search(r'img src="([^"]+)"', text)

    # Check if a match is found
    if match:
        # The text between the quotes is in the first capturing group
        filename = match.group(1)
        #print(filename)
        return filename
    else:
        #print("No match found")
        return ""




In [82]:
# Cadena de ejemplo
text = '<img src=""intro_bob_eq_par_sol_RM.png"">'

# Expresión regular ajustada para las comillas dobles escapadas
filenames = re.findall(r'img src=""([^"]+)""', text)

# Mostrar el resultado
print(filenames)


['intro_bob_eq_par_sol_RM.png']


In [83]:
# Leer el archivo CSV
file_path = full_csv_output  # Cambia por la ruta a tu archivo .csv
print(full_csv_output)


E:\mario\trabajos2\innovación_docente_2024\ankipy\G875.csv


In [84]:
# Definir el número de líneas a saltar (por ejemplo, 6 líneas)
n_lineas_a_ignorar = 6

# Leer el archivo CSV ignorando las primeras N líneas
df = pd.read_csv(file_path, delimiter=';', skiprows=n_lineas_a_ignorar, header=None)



In [85]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,intro.intro.1,What role is each block playing in the followi...,"<img src=""ed_block_diagram_01_mm.png"">",,"<img src=""ed_block_diagram_01_sol_mm.png"">",,,,,,,,,,,#intro,G875::Introduction
1,mecanica.m.1,The inertia of a rotating cilinder depends on:,,mass and radius.,,,,,,,,,,,,#mechanics,G875::Mechanical requirements
2,convertidores.pc.1,"Considering you are in Spain, what is the peak...","<img src=""spectrum_01_mm.png"">",0 V. There is no component of 100 Hz (2nd harm...,,,,,,,,,,,,,G875::Power Converters
3,convertidores.pc.2,What is the mean value of the signal in the fo...,"<img src=""spectrum_01_mm.png"">",5 V,,,,,,,,,,,,,G875::Power Converters
4,convertidores.pc.3,What is the peak value of the following signal?,"<img src=""signal_01_mm.png"">",150 V,,,,,,,,,,,,,G875::Power Converters
5,convertidores.pc.4,"In an ideal power switch, the switching times ...",,[latex]$ t_{on->off} and t_{off->on} $ [/late...,,,,,,,,,,,,,G875::Power Converters
6,convertidores.pc.5,"In an ideal power switch, the equivalent resis...",,[latex]$ R_{on} $ [/latex] equal to 0 ohms.,,,,,,,,,,,,,G875::Power Converters
7,convertidores.pc.6,"In an ideal power switch, the equivalent resis...",,[latex]$ R_{off} -> \infty $[/latex] ohms.,,,,,,,,,,,,,G875::Power Converters
8,convertidores.pc.7,"In an ideal power switch, the power needed to ...",,0 W.,,,,,,,,,,,,,G875::Power Converters
9,convertidores.pc.8,"In an ideal power switch, the maximum voltage ...",,[latex]$ \infty $[/latex] V.,,,,,,,,,,,,,G875::Power Converters


In [86]:



# Extraer todas las apariciones en cada fila
all_filenames = []

for column in df.columns:
    for row in df[column]:
        if isinstance(row, str):
            filenames = extract_filenames(row)
            if len(filenames) > 2:
                print(filenames)
                all_filenames.append(filenames)

# Mostrar los nombres de archivos extraídos
#print("Nombres de archivos extraídos:")
#for filename in all_filenames:
#    print(filename)


ed_block_diagram_01_mm.png
spectrum_01_mm.png
spectrum_01_mm.png
signal_01_mm.png
stepper_motor_01_mm.png
stepper_motor_02_mm.png
ed_block_diagram_01_sol_mm.png


In [87]:
all_filenames

['ed_block_diagram_01_mm.png',
 'spectrum_01_mm.png',
 'spectrum_01_mm.png',
 'signal_01_mm.png',
 'stepper_motor_01_mm.png',
 'stepper_motor_02_mm.png',
 'ed_block_diagram_01_sol_mm.png']

In [88]:
a = 0

if a == 1:

    path = 'C:\\Users\\manan\\Downloads\\AAA\\'  

    # Recorrer la carpeta
    for filename in os.listdir(path):
        # Construir la ruta completa del archivo original
        original_file = os.path.join(path, filename)
    
        # Convertir el nombre del archivo a minúsculas
        new_filename = filename.lower()
    
        # Construir la nueva ruta completa del archivo con el nombre en minúsculas
        new_file = os.path.join(path, new_filename)
    
        # Renombrar el archivo solo si el nombre en minúsculas es diferente
        if original_file != new_file:
            os.rename(original_file, new_file)
            print(f'Renombrado: {filename} -> {new_filename}')
        else:
            print(f'El archivo {filename} ya está en minúsculas')